### This is a notebook to explore build a chatbot via LangChain Liberary

- Use AzureOpenAI's API
- Create prompt template to set the role for AI
- Use memory component to remember the chat context
- Include an agent to support local knowledge QA

### 1 - Create LLM Chat model

In [36]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI 
from langchain.prompts import PromptTemplate

# setup the configuration for LLM
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = "https://qucy-openai-test.openai.azure.com/"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"


In [38]:
# create LLM chat model
llm_normal = AzureOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')
llm_chat = AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')


llm_chat, llm_normal

(AzureChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='chatGPTAzure', openai_api_type='azure', openai_api_version=''),
 AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='gpt-35-turbo', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all', deployment_name='chatGPTAzure'))

### 2 - Create prompt

In [3]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [4]:
knowledge = """
  <knowledge start>
  Things to know
  If the customer wants to open an account, you can offer:
  - Apply via HSBC HK App
  - Book an appointment at the branch online in advance
  - Arrange a call back with International Banking Centre

  Need to open a bank account with HSBC HK? You can apply with us if you:
  - are at least 18 years old
  - meet additional criteria depending on where you live
  - have proof of ID, proof of address

  If customer wants to apply online via mobile app:
  - They need to download the HSBC HK App to open an account online.
  - holding an eligible Hong Kong ID or an overseas passport
  - new to HSBC

  Exchange service:
  Send like a local experience for your global payments
  Our mobile app has enriched the digital experience to cater your global payment needs, with just a few taps on the HSBC HK Mobile Banking App.

  How will it work?
  How to enjoy the fee-free and faster payment experience?
  </knowledge end>
"""

In [5]:
prompt_template = """
You are a customer service for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
You will be proivde some bank knowledge to help you answer questions about HSBC.If they ask non-HSBC related questions, gently remind them you only can provide HSBC HK related knowledge


The following is bank knowledge which you can refer to if it's relevant to the question:
{bank_knowledge}

Chat history only to give you context and don't use any prefix like AI or AI response in the answer:
{chat_history}

Customer Question:
{customer_input}
"""

chat_prompt = PromptTemplate(
    input_variables=["chat_history", "customer_input", "bank_knowledge"], 
    template=prompt_template
)

In [6]:
# get a chat completion from the formatted messages
chat_prompt.format_prompt(bank_knowledge=knowledge, chat_history="", customer_input="How can I open account in HK ?")

StringPromptValue(text="\nYou are a customer service for HSBC in Hong kong. You are always brief and concise in your answers.\nCustomers come to you mostly with questions about account opening procedures but also other HSBC related questions.\nYou will be proivde some bank knowledge to help you answer questions about HSBC.If they ask non-HSBC related questions, gently remind them you only can provide HSBC HK related knowledge\n\n\nThe following is bank knowledge which you can refer to if it's relevant to the question:\n\n  <knowledge start>\n  Things to know\n  If the customer wants to open an account, you can offer:\n  - Apply via HSBC HK App\n  - Book an appointment at the branch online in advance\n  - Arrange a call back with International Banking Centre\n\n  Need to open a bank account with HSBC HK? You can apply with us if you:\n  - are at least 18 years old\n  - meet additional criteria depending on where you live\n  - have proof of ID, proof of address\n\n  If customer wants to 

### 3 - Create memory component

In [7]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryBufferMemory # call LLM to do summarization for the chat histroy

# create window memory - window size = 10 and return messages
memory = ConversationBufferWindowMemory(k=10, memory_key="chat_history", input_key="customer_input")

In [8]:
memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key='customer_input', return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='chat_history', k=10)

### 4 - Chain the LLM chat model and prompt

In [9]:
from langchain import LLMChain

chain = LLMChain(llm=llm_chat, prompt=chat_prompt, memory=memory, verbose=True)

In [10]:
# run
customer_input = "How can I open an account in HK ?"

response = chain.run(customer_input=customer_input, bank_knowledge=knowledge)

print(response)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
You will be proivde some bank knowledge to help you answer questions about HSBC.If they ask non-HSBC related questions, gently remind them you only can provide HSBC HK related knowledge


The following is bank knowledge which you can refer to if it's relevant to the question:

  <knowledge start>
  Things to know
  If the customer wants to open an account, you can offer:
  - Apply via HSBC HK App
  - Book an appointment at the branch online in advance
  - Arrange a call back with International Banking Centre

  Need to open a bank account with HSBC HK? You can apply with us if you:
  - are at least 18 years old
  - meet additional criteria depending on where you live
  - have proof of ID, proof of address

  If cu

In [11]:
customer_input="What material I need to prepare ?"
response = chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
You will be proivde some bank knowledge to help you answer questions about HSBC.If they ask non-HSBC related questions, gently remind them you only can provide HSBC HK related knowledge


The following is bank knowledge which you can refer to if it's relevant to the question:

  <knowledge start>
  Things to know
  If the customer wants to open an account, you can offer:
  - Apply via HSBC HK App
  - Book an appointment at the branch online in advance
  - Arrange a call back with International Banking Centre

  Need to open a bank account with HSBC HK? You can apply with us if you:
  - are at least 18 years old
  - meet additional criteria depending on where you live
  - have proof of ID, proof of address

  If cu

In [12]:
customer_input="Do I need prepare printed version or just digital version ?"
response = chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
You will be proivde some bank knowledge to help you answer questions about HSBC.If they ask non-HSBC related questions, gently remind them you only can provide HSBC HK related knowledge


The following is bank knowledge which you can refer to if it's relevant to the question:

  <knowledge start>
  Things to know
  If the customer wants to open an account, you can offer:
  - Apply via HSBC HK App
  - Book an appointment at the branch online in advance
  - Arrange a call back with International Banking Centre

  Need to open a bank account with HSBC HK? You can apply with us if you:
  - are at least 18 years old
  - meet additional criteria depending on where you live
  - have proof of ID, proof of address

  If cu

### 5 - Scrape knowledge from the public website - https://www.hsbc.com.hk/

In [27]:
website_statement = "We use cookies to give you the best possible experience on our website. By continuing to browse this site, you give consent for cookies to be used. For more details please read our Cookie Policy. close cookie banner Skip page header and navigation Personal Business Global Banking and Markets Global Private Banking List of languages English Selected English 繁體 简体 Log on Log off HSBC Online Banking Register for Personal Internet Banking Stock Trading Business Internet Banking HSBCnet Banking Accounts & Services All-in-One Accounts HSBC Jade HSBC Premier HSBC One See all Employee Banking Solutions See all Deposits Savings Accounts Time Deposits Current Accounts Debit Cards Jade Mastercard® Debit Card Mastercard® Debit Card HSBC Mobile Apps HSBC HK App HSBC HK App - Lite Mode PayMe from HSBC See all Currency & RMB Services 24-hour Currency Exchange Renminbi Services See all Budget See all Well+ See all HSBC Finfit See all Community Banking See all Services & Payments Transfers and payments eCheque Services eStatement and eAdvice MPF See all International Services Open a Hong Kong account Open an overseas account See all Greater Bay Area Services See all Help and support Accessibility View topics Ways to Bank HSBC HK App - Lite Mode Mobile Account Opening HSBC Online Banking Branch Banking Express Banking Phone Banking Important Notices Faster Payment System (FPS) Mobile Cash Withdrawal Mobile Cheque Deposit Digital Hub See all Borrowing Cards & Loans Get a New Credit Card HSBC EveryMile Credit Card HSBC Red Credit Card Visa Signature Card Visa Platinum Card Premier MasterCard HSBC Pulse Dual Currency Card Visa Gold Card for Students Visa Gold Card UnionPay Dual Currency Card See all Overdrafts See all Use Your Credit Card Cards Promotions HSBC Reward+ Mobile Payments HSBC Virtual Credit Card Octopus Auto Top-up Cash Advance Spending Instalment Plan Cash Instalment Plan Interest-free Merchant Instalment Plan Cash Credit Plan See all Mortgages Home Mortgage Loans Home-related Loans Mortgage Repayment Options See all Loans Personal Instalment Loan Personal Instalment Loan Redraw Revolving Credit Facility Balance Consolidation Program Personal Tax Loan Electric Vehicle Personal Instalment Loan Smart Tips on Loans See all Tools & Forms Credit Cards Application Additional Card Application Loans Application Submit documents for credit/debit cards Submit documents for Credit Limit Increase Submit documents for Loans Submit documents for Cash Instalment Plan Instant Mortgage Assessment Property Valuation Tool Form and Document Download Investing Securities & FX Wealth Products Stocks Unit Trusts Bonds/CDs Structured Products Warrants & CBBCs Latest IPOs Gold trading services ESG Investments Compare products See all Wealth Financing See all Wealth Mobile Solutions HSBC Top Trader Club HSBC Trade25 HSBC HK Easy Invest FlexInvest FX Trade FX Your Way Currency Exchange Rates Start Investing Investment Account Opening Wealth Booster Wealth starter guide Investment platform guide Wealth Coach Chat with a Wealth Coach Planning Explore Financial Planning & Wealth Management Future Planner Retirement Wealth Portfolio Intelligence Service HSBC Prism Advisory HSBC Pooled Provident Plan Wealth Insights Our House Views Why ESG Matters Greater Bay Area Services HSBC GBA Wealth Management Connect Quick Links View my Investments Trade Stocks Online Buy Unit Trusts Online Buy Bonds/CDs Online Exchange Currency Online Insurance Protection & Planning Insurance Life Protection Saving & retirement insurance Investment-linked insurance Medical & critical illness insurance Home & helper insurance See all insurance plans Buy Online VHIS Flexi plan Swift Guard Critical Illness Swift Save Family Protector Term Life Home contents insurance Travel insurance See plans you can buy online More from HSBC Life Well+ Benefits+ ORSO LIFE Talk: insurance guide About HSBC Life Already a customer? Submit a claim Manage your policy Forms and documents Investment performance Important information Insights Analysis & Market Data Investment 101 Learn how to invest Learn more about Bonds 5 tips to handle Market Volatility Featured Insights Investment weekly newsletter Our House View Market Outlook Wealth Insights Investment Essentials FX Insights See more Wealth Tool Browse for Funds Bond/CD Screener Stock Quotes Offers Latest Rewards HSBC Latest Offers See more Credit Cards Offers Year-round Privileges RewardCash Rebate Rewards of Your Choice RewardCash Certificate Scheme Instant RewardCash redemption at merchants Mileage Programme RewardCash eShop See more Card Financial Flexibility Spending Instalment Plan Cash Instalment Plan Cash Credit Plan Investment Offers FX Services FlexInvest See more Loans Offers Personal Instalment Loan Accounts Offers HSBC Jade HSBC Premier HSBC One Deposits Mortgages Offers HIBOR-based Mortgage Quick Links Form and Document Download Fees & Charges Website Information Menu Banking Accounts & Services All-in-One Accounts HSBC Jade HSBC Premier HSBC One See all Employee Banking Solutions See all Deposits Savings Accounts Time Deposits Current Accounts Debit Cards Jade Mastercard® Debit Card Mastercard® Debit Card HSBC Mobile Apps HSBC HK App HSBC HK App - Lite Mode PayMe from HSBC See all Currency & RMB Services 24-hour Currency Exchange Renminbi Services See all Budget See all Well+ See all HSBC Finfit See all Community Banking See all Services & Payments Transfers and payments eCheque Services eStatement and eAdvice MPF See all International Services Open a Hong Kong account Open an overseas account See all Greater Bay Area Services See all Help and support Accessibility View topics Ways to Bank HSBC HK App - Lite Mode Mobile Account Opening HSBC Online Banking Branch Banking Express Banking Phone Banking Important Notices Faster Payment System (FPS) Mobile Cash Withdrawal Mobile Cheque Deposit Digital Hub See all Borrowing Cards & Loans Get a New Credit Card HSBC EveryMile Credit Card HSBC Red Credit Card Visa Signature Card Visa Platinum Card Premier MasterCard HSBC Pulse Dual Currency Card Visa Gold Card for Students Visa Gold Card UnionPay Dual Currency Card See all Overdrafts See all Use Your Credit Card Cards Promotions HSBC Reward+ Mobile Payments HSBC Virtual Credit Card Octopus Auto Top-up Cash Advance Spending Instalment Plan Cash Instalment Plan Interest-free Merchant Instalment Plan Cash Credit Plan See all Mortgages Home Mortgage Loans Home-related Loans Mortgage Repayment Options See all Loans Personal Instalment Loan Personal Instalment Loan Redraw Revolving Credit Facility Balance Consolidation Program Personal Tax Loan Electric Vehicle Personal Instalment Loan Smart Tips on Loans See all Tools & Forms Credit Cards Application Additional Card Application Loans Application Submit documents for credit/debit cards Submit documents for Credit Limit Increase Submit documents for Loans Submit documents for Cash Instalment Plan Instant Mortgage Assessment Property Valuation Tool Form and Document Download Investing Securities & FX Wealth Products Stocks Unit Trusts Bonds/CDs Structured Products Warrants & CBBCs Latest IPOs Gold trading services ESG Investments Compare products See all Wealth Financing See all Wealth Mobile Solutions HSBC Top Trader Club HSBC Trade25 HSBC HK Easy Invest FlexInvest FX Trade FX Your Way Currency Exchange Rates Start Investing Investment Account Opening Wealth Booster Wealth starter guide Investment platform guide Wealth Coach Chat with a Wealth Coach Planning Explore Financial Planning & Wealth Management Future Planner Retirement Wealth Portfolio Intelligence Service HSBC Prism Advisory HSBC Pooled Provident Plan Wealth Insights Our House Views Why ESG Matters Greater Bay Area Services HSBC GBA Wealth Management Connect Quick Links View my Investments Trade Stocks Online Buy Unit Trusts Online Buy Bonds/CDs Online Exchange Currency Online Insurance Protection & Planning Insurance Life Protection Saving & retirement insurance Investment-linked insurance Medical & critical illness insurance Home & helper insurance See all insurance plans Buy Online VHIS Flexi plan Swift Guard Critical Illness Swift Save Family Protector Term Life Home contents insurance Travel insurance See plans you can buy online More from HSBC Life Well+ Benefits+ ORSO LIFE Talk: insurance guide About HSBC Life Already a customer? Submit a claim Manage your policy Forms and documents Investment performance Important information Insights Analysis & Market Data Investment 101 Learn how to invest Learn more about Bonds 5 tips to handle Market Volatility Featured Insights Investment weekly newsletter Our House View Market Outlook Wealth Insights Investment Essentials FX Insights See more Wealth Tool Browse for Funds Bond/CD Screener Stock Quotes Offers Latest Rewards HSBC Latest Offers See more Credit Cards Offers Year-round Privileges RewardCash Rebate Rewards of Your Choice RewardCash Certificate Scheme Instant RewardCash redemption at merchants Mileage Programme RewardCash eShop See more Card Financial Flexibility Spending Instalment Plan Cash Instalment Plan Cash Credit Plan Investment Offers FX Services FlexInvest See more Loans Offers Personal Instalment Loan Accounts Offers HSBC Jade HSBC Premier HSBC One Deposits Mortgages Offers HIBOR-based Mortgage Quick Links Form and Document Download Fees & Charges Website Information English English Selected English 繁體 简体 Business Global Banking and Markets Private Banking Warrants & CBBCs Close menu Log on Expanded Collapsed Log off HSBC Online Banking Register for Personal Internet Banking Stock Trading Business Internet Banking HSBCnet Top of main content About our website Read our important documents and notices that covers our website HSBC HK About our Website Privacy and security Learn about our website's privacy and security principles. Terms of use See our website's terms and conditions. Hyperlink policy Read our website's hyperlink policy terms. Accessibility We ensure our website is accessible to customers with different needs. Maintenance schedule Stay up to date with our website maintenance schedule. Important notices Find information relating to our services, terms and conditions and other updates. Regulatory disclosures The following disclosures are made pursuant to section 60A of the Banking Ordinance. Help and support Let us help you find what you need. Sitemap Find the HSBC page that you're looking for. Help and support Contact HSBC Write, call or send a message Find a branch And your nearest Express Banking location Help and Support See FAQs, forms and fees & charges About HSBC Careers, media, investor and corporate information Privacy and Security Terms of Use Online Security Regulatory Disclosures More © Copyright. The Hongkong and Shanghai Ban,king Corporation Limited 2002-2023. All rights reserved This website is designed for use in Hong Kong. Cross-border disclosure"

In [33]:
import requests
from bs4 import BeautifulSoup
import re

homepage = 'https://www.hsbc.com.hk/'

website_statements = re.split('(?<=[.!?]) +', website_statement)

chinese_pattern = re.compile("[\u4e00-\u9fff\u3400-\u4dbf]+")

def scrape_content_by_url(url):
    # Send a GET request to the webpage URL
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract the text content and remove excess whitespace
    text = soup.get_text()
    text = re.sub('\s+', ' ', text).strip().lower()
    text = chinese_pattern.sub("", text)
    for w in website_statements:
        text = text.replace(w.lower(), '')
    return text


def retreive_urls_by_parent_url(url):
    # Send a GET request to the webpage URL
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the url start with a
    a_tags = soup.find_all('a')
    # retrieve all the link
    urls = [a_tag.get('href') for a_tag in a_tags if a_tag is not None and a_tag.get('href') is not None]
    # remove all the link start with http or https
    urls = [url for url in urls if not url.startswith(('http', '#'))]
    # remove other language url
    urls = [url for url in urls if url not in ('/', '/zh-hk/', '/zh-cn/')]
    return urls

def retreive_artical_urls_by_parent_url(url, keywords=['']):
    # Send a GET request to the webpage URL
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the url start with a
    a_tags = soup.find_all('a')
    # retrieve all the link
    urls = [a_tag.get('href') for a_tag in a_tags if a_tag is not None and a_tag.get('href') is not None]
    # only keep all the link start with http
    urls = [url for url in urls if url.startswith(('http'))]
    # if keywords is not empty
    if len(keywords) > 0:
        urls = [url for url in urls if all(keyword in url for keyword in keywords)]
    
    return urls
    

In [40]:
homepage_content = scrape_content_by_url(homepage)
homepage_content

"hsbc hong kong - accounts, insurance, credit cards, loans    close cookie banner skip page header and navigation personal business global banking and markets global private banking list of languages english selected english   log on log off hsbc online banking register for personal internet banking stock trading business internet banking hsbcnet banking accounts & services all-in-one accounts hsbc jade hsbc premier hsbc one see all employee banking solutions see all deposits savings accounts time deposits current accounts debit cards jade mastercard® debit card mastercard® debit card hsbc mobile apps hsbc hk app hsbc hk app - lite mode payme from hsbc see all currency & rmb services 24-hour currency exchange renminbi services see all budget see all well+ see all hsbc finfit see all community banking see all services & payments transfers and payments echeque services estatement and eadvice mpf see all international services open a hong kong account open an overseas account see all grea

In [16]:
sub_urls = retreive_urls_by_parent_url(homepage)
sub_urls[:10]

['/legal/',
 '/jade/',
 '/premier/',
 '/accounts/products/one/',
 '/accounts/',
 '/accounts/employee-banking/',
 '/accounts/products/savings/',
 '/accounts/products/time-deposits/',
 '/accounts/products/current/',
 '/jade/twelve-currency-debit-card/']

In [31]:
legal_url = homepage + sub_urls[0]
legal_content = scrape_content_by_url(legal_url)
legal_content

'about our website - hsbc hk                  '

### 5.1 Retrieve wealth related info

In [59]:
# define url
wealth_insights_pages = [
    'https://www.hsbc.com.hk/en-hk/wealth/insights/wealth-needs/investing-101/',
    'https://www.hsbc.com.hk/wealth/insights/asset-class-views/investment-daily/2023-06-14/',
    'https://www.hsbc.com.hk/wealth/insights/asset-class-views/investment-daily/2023-06-13/',
    'https://www.hsbc.com.hk/wealth/insights/asset-class-views/investment-weekly/2023-06-12/',
    'https://www.hsbc.com.hk/wealth/insights/asset-class-views/investment-monthly/2023-06/',
    'https://www.hsbc.com.hk/wealth/insights/esg/why-esg-matters/2023-06-07/',
    'https://www.hsbc.com.hk/wealth/insights/fx-insights/daily-fx-focus/2023-06-13/',
    'https://www.hsbc.com.hk/en-hk/wealth/insights/trending/why-esg-matter/2019-10/'
]

urls = retreive_artical_urls_by_parent_url('https://www.hsbc.com.hk/wealth/insights/asset-class-views/#investment-daily', ['wealth'])
urls

['https://www.hsbc.com.hk/personal/wealth-management.html',
 'https://www.hsbc.com.hk/en-hk/wealth/insights/asset-class-views/investment-monthly.latest/',
 'https://www.hsbc.com.hk/en-hk/wealth/insights/asset-class-views/investment-monthly.latest/',
 'https://www.hsbc.com.hk/jade/wealth/#exclusive-products-and-preferential-rates/',
 'https://www.hsbc.com.hk/personal/wealth-management.html',
 'https://www.hsbc.com.hk/en-hk/wealth/insights/asset-class-views/investment-monthly.latest/',
 'https://www.hsbc.com.hk/en-hk/wealth/insights/asset-class-views/investment-monthly.latest/',
 'https://www.hsbc.com.hk/jade/wealth/#exclusive-products-and-preferential-rates/']

In [62]:
sub_content = scrape_content_by_url('https://www.hsbc.com.hk/wealth/insights/fx-insights/daily-fx-focus/2023-06-13/')
print(sub_content)

daily fx focus | currency market updates | hsbc hong kong covid-19 measures close we're here to help you manage your finances with our digital banking services on hsbc mobile apps. you can now bank from home with ease and peace of mind. questions about how the coronavirus is affecting your finances or our services? we're here to help.    close cookie banner skip page header and navigation personal business global banking and markets global private banking list of languages english selected english   log on log off hsbc online banking register for personal internet banking stock trading business internet banking hsbcnet banking accounts & services all-in-one accounts hsbc jade hsbc premier hsbc one see all employee banking solutions see all deposits savings accounts time deposits current accounts debit cards jade mastercard® debit card mastercard® debit card hsbc mobile apps hsbc hk app payme from hsbc see all currency & rmb services 24-hour currency exchange renminbi services see all b

### 6 - Save content in the vector store

In [44]:
import faiss

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

# In actual usage, you would set `k` to be a higher value, but we use k=2 to test it
# the vector lookup still returns the semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [46]:
%%time
# save the scraped content into vector store
for url in sub_urls[:10]:
    try:
        full_url = homepage + url
        key_words = url.replace('/', ' ')
        print(full_url)
        # fetch content from internet
        content = scrape_content_by_url(full_url)
        # save into memory
        memory.save_context({"keywords": key_words}, {"content": content})
    except:
        print(f"Skip url {url}")

https://www.hsbc.com.hk//legal/
https://www.hsbc.com.hk//jade/
https://www.hsbc.com.hk//premier/
https://www.hsbc.com.hk//accounts/products/one/
https://www.hsbc.com.hk//accounts/
https://www.hsbc.com.hk//accounts/employee-banking/
https://www.hsbc.com.hk//accounts/products/savings/
https://www.hsbc.com.hk//accounts/products/time-deposits/
https://www.hsbc.com.hk//accounts/products/current/
https://www.hsbc.com.hk//jade/twelve-currency-debit-card/
CPU times: user 1.1 s, sys: 40.8 ms, total: 1.14 s
Wall time: 10.8 s


In [47]:
search_result = memory.load_memory_variables({"prompt": "what is Jade account"})["history"]
search_result[:1000]

'keywords:  jade \ncontent: hsbc jade - hsbc hk    close cookie banner skip page header and navigation personal business global banking and markets global private banking list of languages english selected english   log on log off hsbc online banking register for personal internet banking stock trading business internet banking hsbcnet banking accounts & services all-in-one accounts hsbc jade hsbc premier hsbc one see all employee banking solutions see all deposits savings accounts time deposits current accounts debit cards jade mastercard® debit card mastercard® debit card hsbc mobile apps hsbc hk app hsbc hk app - lite mode payme from hsbc see all currency & rmb services 24-hour currency exchange renminbi services see all budget see all well+ see all hsbc finfit see all community banking see all services & payments transfers and payments echeque services estatement and eadvice mpf see all international services open a hong kong account open an overseas account see all greater bay are

### 7 - Combine knowledge search and LLM chat model

In [48]:
# customer input
customer_input="What benfits can a Jade accout have ?"

# query knowledge
knowledge = memory.load_memory_variables({"customer_input": knowledge})["history"]

# pass the customer input and knowledge to LLM chat model
chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
You will be proivde some bank knowledge to help you answer questions about HSBC.If they ask non-HSBC related questions, gently remind them you only can provide HSBC HK related knowledge


The following is bank knowledge which you can refer to if it's relevant to the question:
keywords:  accounts products one 
content: hsbc one | smarter, all-in-one banking - hsbc hk    close cookie banner skip page header and navigation personal business global banking and markets global private banking list of languages english selected english   log on log off hsbc online banking register for personal internet banking stock trading business internet banking hsbcnet banking accounts & services all-in-one accounts hsbc jade hsbc p

'HSBC Response: With a Jade account, you can enjoy HKD and foreign currency savings and current accounts, time deposits and investments. You will also have a personal HSBC Jade Director to manage your financial goals, access to exclusive investment products, and higher interest rates on time deposits, FX, stocks, bonds, life insurance and more. Additionally, you can benefit from lifestyle management services from Ten Lifestyle Group to plan unforgettable life experiences. To be eligible, you need to maintain a minimum total relationship balance of HKD 7,800,000 or more. Terms and conditions apply.'

In [49]:
# customer input
customer_input="How can I open a Jade account ?"

# query knowledge
knowledge = memory.load_memory_variables({"customer_input": knowledge})["history"]

# pass the customer input and knowledge to LLM chat model
chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
You will be proivde some bank knowledge to help you answer questions about HSBC.If they ask non-HSBC related questions, gently remind them you only can provide HSBC HK related knowledge


The following is bank knowledge which you can refer to if it's relevant to the question:
keywords:  accounts products one 
content: hsbc one | smarter, all-in-one banking - hsbc hk    close cookie banner skip page header and navigation personal business global banking and markets global private banking list of languages english selected english   log on log off hsbc online banking register for personal internet banking stock trading business internet banking hsbcnet banking accounts & services all-in-one accounts hsbc jade hsbc p

'HSBC Response: You can apply for a Jade account via the HSBC HK App, book an appointment at the branch online in advance, or arrange a call back with International Banking Centre. To be eligible, you need to maintain a minimum total relationship balance of HKD 7,800,000 or more. You will also need to provide proof of ID and proof of address. Terms and conditions apply.'